In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from soft_embedding import SoftEmbedding
from dataset_hard import BiasDataset
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd

In [2]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [3]:
### TO CHECK WHETHER THE output_1.logits[:, -1, :] returns the next word logits for each sentence in the batch.

def get_next_word_log_probabilities(model_input_1, model_input_2, model, logsoftmax):
    #import pdb; pdb.set_trace()
    output_1 = model(input_ids = model_input_1[0],attention_mask = model_input_1[1])     
    #here we get the logits from the output, the logits are obtained from the output.logits, here we first get
    #the [batch_size, sequence_len, vocab_size]
    #next_word_logits_1 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_1.logits)])
    next_word_logits_1 = output_1.logits[:, -1, :]
    next_word_log_probs_1 = logsoftmax(next_word_logits_1)
    
    output_2 = model(input_ids = model_input_2[0],attention_mask = model_input_2[1] ) 
    #next_word_logits_2 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_2.logits)])
    next_word_logits_2 = output_2.logits[:, -1, :]
    next_word_log_probs_2 = logsoftmax(next_word_logits_2)    

    return next_word_log_probs_1, next_word_log_probs_2


In [4]:
n_tokens = 5
initialize_from_vocab = True
num_epochs = 1

In [5]:
dataset = BiasDataset("../data/occupation.csv", n_tokens = 5)
dataloader = dataset.get_dataloader(batch_size=64)

#dataloader = DataLoader(dataset = dataset, batch_size = 4,  shuffle = False, num_workers=1, collate_fn = collate_fn)

In [6]:
for data in dataloader:
    print(data)

((tensor([[ 3987,   470,   307, 24651, 22427,   550,   257,  8507,   329, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 22427,   373,  5863,   329, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  2219,  3725,   326, 22427,
         43748,  3353,   287, 50256, 50256],
        [ 3987,   470,   307, 24651, 22427,   373,  8018,   329, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380,  3917, 22427,   351, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 22427,   550,   257,  2610,  1700,   286,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 22427,   373, 10810,   329, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 22427,   550,   257,  7310, 46283,   329,
         50256, 50256, 50256, 50256, 50256],
      

((tensor([[ 3987,   470,   307, 24651,  4380,  8018,  4257,   329,   852, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4257,   373, 13943,   329,   852, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,   880,    12, 27718,   326,
          4257,   373, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380, 10810,  4257,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4257,   373, 48243,   329,   852, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  4143,  6292,   326,  4257,
           373, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380,  5174,  4257,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4257,   373, 10810,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  6768,  476

((tensor([[ 3987,   470,   307, 24651,   632,   373,  6768,  6292,   290,  8018,
           326,  2988,   373, 50256],
        [ 3987,   470,   307, 24651,  4380,   550,   257,  1598,  4547,   286,
          2988,   852, 50256, 50256],
        [ 3987,   470,   307, 24651,  2988,   373, 13943,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  6768,  5385,   326,  2988,
           373, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380,  5174,  2988,   329,   852, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  2988,   373,  8811,  6412,   284,   355,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,   880,    12, 27718,   326,
          2988,   373, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380,   547,  5385,   351,  2988,   329,
           852, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  2988,   373,  6768,  801

((tensor([[ 3987,   470,   307, 24651,   383, 25230,   286, 13850,   373,  6768,
          8018,   287, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 13850,   550,   281, 40016, 29033,  8507,
           329, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   383, 13572,   286, 13850,   373, 22140,
           287, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  6768, 10810,   326, 13850,
           550,   257, 37457,   329, 50256],
        [ 3987,   470,   307, 24651,   383,  1438,   286, 13850,   373,  5174,
           351, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 13850,   373,   880,    12,  4002,   355,
           257,  3554,   287, 50256, 50256],
        [ 3987,   470,   307, 24651,   383,  8507,   286, 13850,   373,  1912,
           319, 50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651, 13850,   550,   257,  6414,  8507,   329,
         50256, 50256, 50256, 50256, 50256],
      

((tensor([[ 3987,   470,   307, 24651,   632,   373,  4143,  7247,   326,  3956,
           373, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   383,   835,   661,  6619,   546,  3956,
          8203,   326,   340,   373],
        [ 3987,   470,   307, 24651,  4380,  8018,  3956,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  8811, 10810,   326,  3956,
           373, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   383,  3722,   286,  3956,   373, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  4762,   416,   867,   326,
          3956,   373, 50256, 50256],
        [ 3987,   470,   307, 24651,   383,  2292,   286,  3956,   373, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,  4380, 48243,  3956,   355, 50256, 50256,
         50256, 50256, 50256, 50256],
        [ 3987,   470,   307, 24651,   632,   373,  6768,  476

In [7]:
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)
logsoftmax = nn.LogSoftmax(dim = 0)
kl_loss = nn.KLDivLoss(reduction="batchmean", log_target=True)

In [8]:
#loss before training

In [9]:
loss = 0
curr_loss = []
for iter, data in enumerate(dataloader):
    group_1, group_2 = data[0], data[1]
    with torch.no_grad():
        log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
        loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
        curr_loss.append(loss)
    print(f'loss at batch : {iter} is {loss.item()}')

torch.mean(torch.stack(curr_loss))

loss at batch : 0 is 0.02570141665637493
loss at batch : 1 is 0.0016912759747356176
loss at batch : 2 is 0.0005022574914619327
loss at batch : 3 is 0.0028203583788126707
loss at batch : 4 is 0.017165295779705048
loss at batch : 5 is 0.009527482092380524
loss at batch : 6 is 0.003926165867596865
loss at batch : 7 is 0.0023539175745099783
loss at batch : 8 is 3.991366975242272e-05
loss at batch : 9 is 0.0012809617910534143
loss at batch : 10 is 0.000845477741677314
loss at batch : 11 is 0.001107388292439282
loss at batch : 12 is 0.0008672429248690605
loss at batch : 13 is 0.010490315966308117
loss at batch : 14 is 0.005286047235131264
loss at batch : 15 is 0.025058938190340996
loss at batch : 16 is 0.009242793545126915
loss at batch : 17 is 0.011552257463335991
loss at batch : 18 is 0.008487289771437645
loss at batch : 19 is 0.00035078637301921844
loss at batch : 20 is 0.0011522878194227815
loss at batch : 21 is 0.0006331499316729605
loss at batch : 22 is 0.0006960268365219235
loss at ba

tensor(0.0054)

In [1]:
0.015

0.015